# Game of life

### Errors i correccions:


+ Funció range(-1,1) del Drapi estava malament (només generava la llista [-1,0]), per tant l'error que saltava al programa del Llorenç era correcte.<br>
> S'ha corretgit posant range(-1,2) que genera la llista [-1,0,1]. Línies 12 i 13 del la segona cel·la

+ Per eliminar l'error de l'índex <em>out of bounds</em> he ignorat la fila i columna 25.<br>
> S'ha corretgit substituint range(len(mapa)) per range(len(mapa)-1) en els dos <b>for</b> generals. Línies 21 i 22 de la tercera cel·la 

+ El programa anava fent el checkBoundaries en cel·les que just havien canviat, quan ho hauria d'haver calculat tot sense canviar res fins al final.
> Corretgit creant un mon "estatic" que no canvia els valors fins haver recorregut tota la matriu. 

+ Algortime propi de the Game of Life no era exactament el que havíem escrit amb el<b> if, else </b>i creava massa vida
> Corretgit canviant i afegint dos <b>if</b> a les linies 25-28, que quan tenen dos veins, la cel·la es queda igual


### Extres:
+ He afegit una animació. Això es fa amb el mòdul d'animació i generant 60 imatges (Ho fa el <b>for</b> de la línia 16 de la tercera cel·la)


+ Afegida una aplicació feta amb la llibreria GTK+ que permet més coses xules

<br>

### Llibreries per l'animació estàtica

In [1]:
#importar llibreries per animacio

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation
from IPython.display import HTML

#%matplotlib gtk

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### Llibreries per l'aplicació dinàmica

In [1]:
#importar llibreries per aplicacio

import gi
import numpy as np

gi.require_version('Gtk', '3.0')
from gi.repository import Gtk
from gi.repository import GLib

### Funcions primàries

In [2]:
def vida(mapa,factorvida):
    for i in range(len(mapa)):
        for j in range(len(mapa[i])):
            randomnumber = np.random.rand()
            if randomnumber > factorvida:
                mapa[i][j] = 1
            else:
                mapa[i][j] = 0

def checkboundaries(mapa,x,y):
    veins= 0
    for i in range(-1,2):
        for j in range(-1,2):
            if i == 0 and j == 0:
                pass
            else:
                vei = mapa[x+i][y+j]
                veins = veins + vei 
    return veins

def comprovarvida(mapa,i,j):
    veinspropers = checkboundaries(mapa,i,j)
    if veinspropers > 3 or veinspropers < 2:
        return 0
    if veinspropers == 3:
        return 1
    if veinspropers == 2:
        return mapa[i][j]

def reproductor(mapa):
    oldMapa = mapa
    mapa = np.zeros((len(mapa),len(mapa)))

    for i in range(len(oldMapa)-1):
        for j in range(len(oldMapa[i])-1):
            mapa[i][j] = comprovarvida(oldMapa,i,j)
            
    return mapa

### Definició de l'aplicació GTK3+

In [4]:
class GameOfLife(Gtk.Window):

    def __init__(self):
        Gtk.Window.__init__(self, title="El Xoc es un Bon Gos")
        Gtk.Window.set_default_size(self, 400,400)

        reproduir = Gtk.Button(label="Saltar")
        reproduir.connect("clicked", self.saltar)
        
        random = Gtk.Button(label="Randomitzar")
        random.connect("clicked", self.randomitzar)
        
        reproduccio = Gtk.ToggleButton(label="Reproduir")
        reproduccio.connect("toggled", self.reproduir)
        
        borrar = Gtk.Button(label="Reset")
        borrar.connect("clicked", self.delete)
        
        vbox = Gtk.VBox()
        box = Gtk.Box()
        box.add(reproduir)
        box.add(random)
        box.add(reproduccio)
        box.add(borrar)
        vbox.pack_start(box, False, False, 2)
        
        self.grid = Gtk.Grid()
        vbox.pack_end(self.grid, True, True, 2)
        self.add(vbox)

    def setGrid(self, x, y):
        self.x = x
        self.y = y
        self.mapa = np.zeros((x,y))
        for i in range(x):
            for j in range(y):
                button = Gtk.ToggleButton(label="")
                button.connect("toggled", self.clickCella, j,i)
                self.grid.attach(button, j,i,1,1)
                
    def setVida(self, v):
        self.vida = v
        
    def setVelocitat(self, v):
        self.velo = v
        
    def clickCella(self, boto, i, j):
        if boto.get_active():
            self.mapa[i][j] = 1
        else:
            self.mapa[i][j] = 0
            
    def calcular(self):
        self.mapa = reproductor(self.mapa)
        for i in range(len(self.mapa)):
            for j in range(len(self.mapa)):
                boto = self.grid.get_child_at(i,j)
                if self.mapa[i][j] == 1:    
                    boto.set_active(True)
                else:
                    boto.set_active(False)
            
    def randomitzar(self, boto):
        vida(self.mapa, self.vida)
        for i in range(len(self.mapa)):
            for j in range(len(self.mapa)):
                boto = self.grid.get_child_at(i,j)
                if self.mapa[i][j] == 1:    
                    boto.set_active(True)
                else:
                    boto.set_active(False)
        
    def reproduir(self, boto):
        if boto.get_active():
            self.calcular()
            GLib.timeout_add(self.velo, self.reproduir, boto)
        
    def saltar(self, boto):
        self.calcular()
    
    def delete(self, boto):
        self.mapa = np.zeros((self.x, self.y))
        for i in range(len(self.mapa)):
            for j in range(len(self.mapa)):
                boto = self.grid.get_child_at(i,j)
                if self.mapa[i][j] == 1:    
                    boto.set_active(True)
                else:
                    boto.set_active(False)

# Animació amb el Matplotlib

In [3]:
#25x25 zeros
world = np.zeros((25, 25))

#25x25 zeros o uns
vida(world,0.6)

#configurar el color del mapa
my_cmap = cm.get_cmap('viridis')
my_cmap.set_under('w')
fig = plt.figure(figsize=(10,10))

#crear 60 imatges
ims = []

for i in range(60):
    world = reproductor(world)
    im = plt.imshow(world, interpolation="nearest", cmap=my_cmap, animated=True)
    ims.append([im])

#animar les 60 imatges
ani = animation.ArtistAnimation(fig, ims, interval=120, blit=True,repeat_delay=500)
HTML(ani.to_html5_video())

# Aplicació feta amb GTK

In [5]:
gof = GameOfLife()
gof.setGrid(30, 30)
gof.setVida(0.75)
gof.setVelocitat(150)
gof.connect("delete-event", Gtk.main_quit)
gof.show_all()

Gtk.main()